In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

# Process a single frame for person detection (tracking not included)
def process_frame(frame, model):
    #frame actually a picture
    results = model(frame)
    detections = []

    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        conf = float(box.conf)
        cls = int(box.cls)

        # class 0 = person in COCO
        if cls == 0 and conf > 0.5:
            detections.append([[float(x1), float(y1), float(x2 - x1), float(y2 - y1)],conf])
    
    return detections


# now, detections is a list of list like [[x, y, w, h]] which is the information of bounding boxes around the person detected in the frame.

In [ ]:
### validation of the process_frame function ###

import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Function to process frame (detect persons only)
def process_frame(frame, model):
    results = model(frame)
    detections = []

    for box in results[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        conf = float(box.conf)
        cls = int(box.cls)

        # class 0 = person in COCO dataset
        if cls == 0 and conf > 0.5:
            detections.append([float(x1), float(y1), float(x2 - x1), float(y2 - y1)])  # [x, y, w, h]

    return detections


def main():

    image_path = "20.jpg"   
    frame = cv2.imread(image_path)
    frame = cv2.resize(frame, (640, 480))

    if frame is None:
        print("Error: Image not found!")
        return

    detections = process_frame(frame, model)
    print("Detections:", detections)

    # Draw green rectangles for each detection
    for (x, y, w, h) in detections:
        x, y, w, h = int(x), int(y), int(w), int(h)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the image
    cv2.imshow("Person Detections", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()



0: 480x640 16 persons, 68.1ms
Speed: 2.5ms preprocess, 68.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Detections: [[175.90618896484375, 283.386474609375, 59.803466796875, 140.6126708984375], [22.92767333984375, 247.980224609375, 55.08995819091797, 135.06640625], [336.8275146484375, 20.076690673828125, 44.910888671875, 122.70515441894531], [269.208740234375, 222.44500732421875, 54.98626708984375, 134.27813720703125], [459.22979736328125, 239.7570343017578, 64.21868896484375, 123.30662536621094], [235.919921875, 138.51385498046875, 42.9990234375, 115.30282592773438], [302.77349853515625, 117.91400146484375, 46.9053955078125, 122.28732299804688], [95.42851257324219, 94.6858901977539, 44.63526916503906, 118.70928192138672], [131.28701782226562, 134.75360107421875, 45.83489990234375, 117.2784423828125], [189.84017944335938, 82.3543701171875, 48.1597900390625, 165.18399047851562], [71.75164794921875, 6.6855316162109375, 34.08937072753906, 105.83248901367188], [164.8

In [2]:
### anomalies detection function ###
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model


def anomaly_detection(detections, frame):
    def focal_loss(gamma=2., alpha=0.25):
        def focal_loss_fixed(y_true, y_pred):
            y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())  # numerical stability
            cross_entropy = -y_true * K.log(y_pred)
            loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
            return K.sum(loss, axis=1)
        return focal_loss_fixed
    
    model = load_model('densenet_focal_epoch2.h5',custom_objects={'focal_loss_fixed': focal_loss(gamma=2.0, alpha=0.25)})
    for (x, y, w, h) in detections:
        x, y, w, h = int(x), int(y), int(w), int(h)
        cropped = frame[y:y+h, x:x+w]
        cropped = cv2.resize(cropped, (64, 64))
        cropped_array = image.img_to_array(cropped)
        cropped_array = np.expand_dims(cropped_array, axis=0)  # Add batch dimension
        cropped_array /= 255.0

        prediction = model.predict(cropped_array)
        prdiction_class = np.argmax(prediction, axis=1)[0]
        if prdiction_class == 7:
            return False
        else:
            return True  # Anomaly detected  
    return False  # No anomaly

In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort
import torchreid

tracker = DeepSort(max_age=90,embedder="torchreid",
    embedder_model_name="resnet50",  
    half=True  )

def tracking(detections, frame):
    dets = []
    for (x, y, w, h) in detections:
        x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)
        dets.append(([x1, y1, x2, y2], 1.0))  

    tracks = tracker.update_tracks(dets, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return frame